In [1]:
from flask import Flask, jsonify, request
import pandas as pd
import joblib
import numpy as np

In [2]:
def map_dispositivo(dispo):
    if type(dispo) == str:
        if 'xiaomi' in dispo:
            return 'xiaomi phone'
        elif 'motorola' in dispo:
            return 'motorola phone'
        elif 'oppo' in dispo:
            return 'oppo phone'
        elif 'ipad' in dispo:
            return 'apple ipad'
        elif 'huawei' in dispo:
            return 'huawei phone'
        elif 'samsung galaxy' in dispo:
            return 'samsung galaxy phone'
        elif 'samsung samsung' in dispo:
            # dejo esto como otra categoria porque los galaxy suelen ser mas premium
            return 'samsung no galaxy phone'
        elif 'lg' in dispo:
            return 'lg phone'
        elif 'sony' in dispo:
            return 'sony phone'
        elif 'lenovo' in dispo:
            return 'lenovo device'
        elif 'bq' in dispo:
            return 'bq phone'
        elif 'tcl' in dispo:
            return 'tcl device'
        elif 'realme' in dispo:
            return 'realme device'
        elif 'asus' in dispo:
            return 'asus device'
        elif 'oneplus' in dispo:
            return 'oneplus device'
        elif dispo == 'apple iphone':
            return dispo
        else:
            return 'otros'

def llenar_missings_ciudad(pais, region):
    return ciudad_mas_frecuente_por_pais[pais]        

def setear_max(df, col, nuevo_max):
    df[col] = df.apply(lambda x: nuevo_max if x[col] > nuevo_max else x[col], axis = 1)
    return df

def transformar_ciudad_pais(pais, ciudad):
    if ciudad in ciudades_populares:
        return ciudad
    elif pais in paises_populares:
        return pais
    else:
        return "lugar_poco_frecuente"

def solo_una_cifra_despues_del_punto(version_os):
    version_split = version_os.split('.')
    if len(version_split) > 2:
        version_os = version_split[0] + '.' + version_split[1]
    return version_os

model = joblib.load('model/modelo_GBR.pkl')
ciudad_mas_frecuente_por_pais = joblib.load('model/moda_por_pais.pkl')
percentil_99 = joblib.load('model/percentil_99.pkl')
os_version_con_menor_frec = joblib.load('model/os_version_con_menor_frec.pkl')
std = joblib.load('model/standarizer.pkl')
country_region_con_menor_frec = joblib.load('model/country_region_con_menor_frec.pkl')

moda_device_family = 'apple iphone'        
moda_source = 'Organic'
moda_event_1, moda_event_2, moda_event_3 = 0, 0, 0
paises_populares = ['es', 'fr', 'mx', 'ar', 'us', 'co', 'it']
ciudades_populares = ['Madrid', 'Barcelona', 'Valencia', 'Seville', 'Paris', 'Malaga', 'Mexico City', 'Alicante', 
                      'Biscay', 'Murcia', 'Las Palmas', 'Pontevedra', 'Balearic Islands', 'Valladolid', 'Granada', 
                      'Andalusia', 'Cadiz']
columnas_modelo = ['source', 'place', 'platform', 'device_family', 'os_version', 'event_1', 'event_2', 'event_3']

tipos_de_version_os = ['10', '11', '12.5', '14.2', '14.3', '14.4', '6.0', '7.0', '7.1', '8.0', '8.1', '9', 'otra_version']

tipos_de_dispositivo = ['apple ipad', 'apple iphone', 'asus device', 'bq phone', 'huawei phone', 'lenovo device', 'lg phone', 
'motorola phone', 'oneplus device', 'oppo phone', 'otros', 'realme device', 'samsung galaxy phone', 
'samsung no galaxy phone', 'sony phone', 'tcl device', 'xiaomi phone']

tipos_de_os = ['android', 'ios']

lugares_posibles = ['Alicante', 'Andalusia', 'Balearic Islands', 'Barcelona', 'Biscay', 'Cadiz', 'Granada', 
                    'Las Palmas', 'Madrid', 'Malaga', 'Mexico City', 'Murcia', 'Paris', 'Pontevedra', 'Seville', 
                    'Valencia', 'Valladolid', 'ar', 'co', 'es', 'fr', 'it', 'lugar_poco_frecuente', 'mx', 'us']

cols = ['10', '11', '12.5', '14.2', '14.3', '14.4', '6.0', '7.0', '7.1', '8.0', '8.1', '9', 'otra_version', 
        'apple ipad', 'apple iphone', 'asus device', 'bq phone', 'huawei phone', 'lenovo device', 'lg phone', 
        'motorola phone', 'oneplus device', 'oppo phone', 'otros', 'realme device', 'samsung galaxy phone', 
        'samsung no galaxy phone', 'sony phone', 'tcl device', 'xiaomi phone', 'android', 'ios', 'Alicante', 
        'Andalusia', 'Balearic Islands', 'Barcelona', 'Biscay', 'Cadiz', 'Granada', 'Las Palmas', 'Madrid', 
        'Malaga', 'Mexico City', 'Murcia', 'Paris', 'Pontevedra', 'Seville', 'Valencia', 'Valladolid', 
        'ar', 'co', 'es', 'fr', 'it', 'lugar_poco_frecuente', 'mx', 'us', 'Non-Organic', 'Organic', 'event_1', 
        'event_2', 'event_3']

In [ ]:
app = Flask(__name__)
#vamos a tomar el input con el json con los datos que al que le tenemos que dar una prediccion, pasarlo por el 
#proceso que pasamos los datos en el notebook donde generamos el modelo para luego hacer la prediccion y devolverla
@app.route("/predict", methods=['POST'])
def do_prediction():
    json = request.get_json()
    df = pd.DataFrame(json, index=[0])

    for col in df.columns:
        if df[col].dtype == 'O' and col != 'user_id':
            df[col] = df[col].str.strip()
    if "event_3" not in df.columns:
        df['event_3'] = 0
    if "country_region" not in df.columns:
        df['country_region'] = 'Madrid'
    if "source" not in df.columns:
        df['source'] = 'Organic'
    if "device_family" not in df.columns:
        df['device_family'] = moda_device_family
    if "os_version" not in df.columns:
        df['os_version'] = '10'
                
    df['platform'] = df['platform'].str.lower()
    
    df['device_family'] = df['device_family'].str.lower()
    df['device_family'] = df['device_family'].apply(lambda x : map_dispositivo(x))
    df['device_family'] = df['device_family'].fillna(moda_device_family)
    
    df['source'] = df['source'].fillna(moda_source)
    
    for col in ['event_2','event_3']:
        df = setear_max(df, col, percentil_99[col])
    df = setear_max(df, col, percentil_99['revenue'])
    
    df['event_1'] = df['event_1'].apply(lambda x: 1 if x > 0 else 0)
    
    df['os_version'] = df['os_version'].apply(lambda x: solo_una_cifra_despues_del_punto(x))
    df.loc[df["os_version"].isin(os_version_con_menor_frec.index.tolist()), 'os_version']= "otra_version"
    
    df['country_region'] = df.apply(lambda x: llenar_missings_ciudad(x['country'], x['country_region']) if not pd.notnull(x['country_region']) else x['country_region'], axis = 1)
    df.loc[df["country_region"].isin(country_region_con_menor_frec.index.tolist()), 'country_region']= "otras_ciudades"    
    df['place'] = df.apply(lambda x: transformar_ciudad_pais(x['country'], x['country_region']), axis = 1)
    
    df = df[columnas_modelo]
    
    vector_para_predecir = []
    
    vector_version_os = [int(os_version == df['os_version']) for os_version in tipos_de_version_os]
    if 1 not in vector_version_os:
        # si es un os que no tenemos registrado va a la cateogia de otros
        vector_version_os[-1] = 1

    vector_tipos_de_dispositivo = [int(disp == df['device_family']) for disp in tipos_de_dispositivo]
    if 1 not in vector_tipos_de_dispositivo:
        vector_tipos_de_dispositivo[10] = 1 # 10 es el indice de otros en el vector

    vector_os = [int(os == df['platform']) for os in tipos_de_os]
    if 1 not in vector_os:
        vector_os[0] = 1 #la moda son android, no deberia pasar que no sean andriod ni ios, aca habria que registrar un error

    vector_lugares_posibles = [int(lugar == df['place']) for lugar in lugares_posibles]
    if 1 not in vector_lugares_posibles:
        vector_lugares_posibles[22] = 1 # 22 es el indice de lugar_poco_frecuente

    vector_organic = [int(organic == df['source']) for organic in ['Non-Organic', 'Organic']]

    vector_para_predecir += vector_version_os + vector_tipos_de_dispositivo + vector_os + vector_lugares_posibles + vector_organic + [df['event_1'][0]] +[df['event_2'][0]] + [df['event_3'][0]]

    fila_a_predecir = pd.DataFrame([vector_para_predecir], columns=cols)

    df_x_scalado = std.transform(fila_a_predecir)
    
    revenue_estimado = model.predict(df_x_scalado)

    result = {"revenue_estimado" : revenue_estimado[0]}
    return jsonify(result)

if __name__ == "__main__":
    app.run(host='0.0.0.0')

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://0.0.0.0:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [20/May/2021 23:18:42] "POST /predict HTTP/1.1" 200 -
